In [2]:
# Import Core Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Set Theme for Plotly Maps
# Different MapBox Styles
# carto-darkmatter
# open-street-map
# carto-positron

# Set Global Map Theme
map_theme = 'carto-darkmatter'

# Set Theme for graphs
sns.set_theme(palette='bright')

# Set Pandas Parameters
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

# Import DataFrame
stl_df = pd.read_csv('../Data/STL-Data-2023.csv')

# Create County DataFrame
stl_county_df = stl_df.loc[stl_df['Geo-County'] == 'Saint Louis County']

# Create City DataFrame
stl_city_df = stl_df.loc[stl_df['Geo-County'] == 'Saint-Louis City']

## Build Directory Structure

In [5]:
# Create Directories for City & County
def createCityCountyDirectores(nameList):
    '''
    Title: Create City & County Directories
    Description: This will create the City & County Diretories.
    nameList: This the list of names for directories to create.
    '''

    for name in nameList:
        # Set full path to a variable
        fullPath = name + '/'

        dirExists = os.path.exists(fullPath)

        # Check if directory already exists
        if dirExists == False:
            # Create directory
            os.mkdir(fullPath)
        else:
            pass

# Create Directory Structure for any Area
def createAreaDirectories(dataFrame, areaColumn, areaPath):
    '''
    Title: Create Area Directories
    Description: This will create seperate directories for each area in areaColumn.
    dataFrame: The dataframe of data that the area columns are in.
    areaColumn: The specfic column of places that need directories created.
    area_path: The path in which you would like the directories created in.
    '''

    # Create list of all area names
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    bigDirExists = os.path.exists(areaPath)

    # Check if directory already exists
    if bigDirExists == False:
        # Create directory
        os.mkdir(areaPath)
    else:
        pass

    for area in area_list:

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set full path to a variable
        fullPath = os.path.join(areaPath,area)

        # Directory boolean
        littleDirExists = os.path.exists(fullPath)

        # Check if directory already exists
        if littleDirExists == False:
            # Create directory
            os.mkdir(fullPath)
        else:
            pass


# Create Folders for City and County
stl_name_list = ['City','County', 'Patrol Districts']

# Create City & County Directories
createCityCountyDirectores(nameList=stl_name_list) # Name Lists to create directories

# Create a Folder for each Town
createAreaDirectories(dataFrame=stl_df,        # DataFrame
                      areaColumn='Geo-Town',   # Area Column to iterate through                                     
                      areaPath='Towns/')       # Folder to put areas in

# Create a Folder for each Village
createAreaDirectories(dataFrame=stl_df,            # DataFrame                
                      areaColumn='Geo-Village',    # Area Column to iterate through                    
                      areaPath='Villages/')        # Folder to put areas in                        

# Create a Folder for each Neighborhood
createAreaDirectories(dataFrame=stl_df,                # DataFrame                    
                      areaColumn='Geo-Neighborhood',   # Area Column to iterate through                      
                      areaPath='Neighborhoods/')       # Folder to put areas in                    

# Create Folders for Patrol Districts
createAreaDirectories(dataFrame=stl_df,                    # DataFrame           
                      areaColumn='Nei-Patrol-District',    # Area Column to iterate through                                  
                      areaPath='Patrol Districts/')        # Folder to put areas in                     

## Patrol Districts

### Patrol Area Pictures

In [ ]:
def createPatrolPictures(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Patrol Pictures
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: The column name of the type of nibrs information that would be mapped.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create dictionary
        area_vc_mappings = dict()

        # Reverse the order of dataframe
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{areaColumn} Crime Points',   # Chart Name
                             lat=area_latitudes,                                  # Latitudes
                             lon=area_longitudes,                                 # Longitudes
                             showlegend=False,                                    # Don't show legends
                             marker=dict(color=area_df['NIBRS-Map'])))            # Hover Information Template     

        # Create Layout
        mapPlot.update_layout(height= 900,                                                # Height of Subplot
                              width= 1400,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        
        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        coverImagePath = f'CoverImage-2023-{area}.jpeg'
        areaFullPath = f'{areaPath}{area}'

        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save Image
        mapPlot.write_image(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Pictures Created for {areaColumn}!')

# Create Cover Images for each Town, Village & Neighborhood
createPatrolPictures(dataFrame=stl_df,                         # DataFrame
                     areaColumn='Nei-Patrol-District',         # Column of Area
                     nibrsColumn='NIBRS-Offense',              # NIBRS Crime Column
                     zoomLevel=11.5,                           # Zoom Level        
                     areaPath='Patrol Districts/')             # Path to directory

### Patrol Scatter Maps

In [13]:
def createAreaScatterMap(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Scatter Map
    Description: This function creates a scatter map of all the latitudes and longitudes
    correlating to the color of how many crime points there are.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())
        area_vc_mappings = dict()
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)

        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'

        # Create Scatter Map Base Plot
        scatterMap = go.Figure(go.Scattermapbox())

        # Create list of Colors that correlate to continuous color values like other charts
        markerColors = px.colors.sample_colorscale('plasma',                                             # Color palette list
                                                   samplepoints=area_vc['count']/area_vc['count'].max())   # The range of point from min-max

        # Create Counter
        i=0

        # Add each unique NIBRS column value to the scatter map in correlating colors
        for nibrsName in area_vc[nibrsColumn].unique():
            
            # Create that nibrs column value as it owns dataframe
            nibs_df = area_df[area_df[nibrsColumn] == nibrsName]
            
            # Create each trace of Points
            scatterMap.add_trace(go.Scattermapbox(visible=True,                                             # Make sure all points are visible
                                                  name=nibrsName,                                           # Value of NIBRS for legend corrlation
                                                  mode="markers",                                           # Set mode to markers
                                                  lat=nibs_df['Geo-Latitude'],                              # Latitude Points
                                                  lon=nibs_df['Geo-Longitude'],                             # Longitude Points
                                                  marker=dict(color=f'{markerColors[i]}'),                  # Set the color of each point
                                                  customdata=nibs_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                                  hovertemplate="<br>".join([area_hover, nibrs_hover])))    # Hover value structure
            
            # Add to counter
            i = i+1

        # Update ScatterMap Layout
        scatterMap.update_layout(height= 750,                                                   # Height of Subplot
                                 width= 1300,                                                   # Width of Subplot
                                 margin={"r":0,"t":0,"l":0,"b":0},                              # Margin values right, top, left, bottom                  
                                 autosize=True,                                                 # Set Autosize of charts to True or False
                                 template='plotly_dark',                                        # Template of Subplot
                                 legend=dict(title=f'{nibrsColumn}'),                           # Legend title
                                 mapbox=dict(zoom=zoomLevel,                                    # Zoom of Map
                                                pitch=0,                                        # Pitch of Map
                                                bearing=0,                                      # Bearing of Map
                                                style=map_theme,                                # Theme of Scatter Map
                                                center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                            lon=np.mean(area_longitudes))))     # Mean of Area Longitudes    # Mean of Area Longitudes
        
        # Add Geographical Data Quality Annotation
        scatterMap.add_annotation(dict(y=0.01,                                             # y position 
                                       x=0.01,                                             # X position 
                                       yref="paper",                                       # set y reference to paper
                                       xref="paper",                                       # Set X reference to paper
                                       xanchor='left',                                     # Set x anchor
                                       showarrow=False,                                    # Don't show arrow
                                       textangle=0,                                        # Angle of Title
                                       font=dict(color='grey',                             # Font color
                                                 size=8),                                  # Font size
                                       text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
    

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMap-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterMap.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn}!')

# Create Scatter Maps
createAreaScatterMap(dataFrame=stl_df,                   # DataFrame
                     areaColumn='Nei-Patrol-District',   # Area Column
                     nibrsColumn='NIBRS-Offense',        # NIBRS Column
                     zoomLevel=11.5,                     # Zoom Level value
                     areaPath='Patrol Districts/')       # Directory location  

createAreaScatterMap(dataFrame=stl_df,                       # DataFrame
                     areaColumn='Nei-Patrol-District',       # Area Column
                     nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column
                     zoomLevel=11.5,                         # Zoom Level value
                     areaPath='Patrol Districts/')           # Directory location                 

[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District!
[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District!


In [5]:
def createPatrolDistrictScatterMap(dataFrame, patrolColumn, zoomLevel, areaPath):
    '''
    Title: Create Patrol District Scatter Maps
    Description: This function creates a scatter map of all the latitudes and longitudes
    correlating to the color of how many crime points there are.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    stl_patrol_df = dataFrame

    # Value Count Lists
    patrol_vc = pd.DataFrame(stl_patrol_df[patrolColumn].value_counts().reset_index())

    # Create Scatter Map Base Plot
    scatterMap = go.Figure(go.Scattermapbox())

    # Create list of Colors that correlate to continuous color values like other charts
    markerColors = px.colors.sample_colorscale('plasma',                 # Color palette list
                                                samplepoints=[1,.5,0])   # The range of point from min-max

    # Create Counter
    i=0

    # Add each unique NIBRS column value to the scatter map in correlating colors
    for patrolName in patrol_vc[patrolColumn].unique():
        
        # Create that nibrs column value as it owns dataframe
        patrol_District_df = stl_patrol_df[stl_patrol_df[patrolColumn] == patrolName]
        
        # Create each trace of Points
        scatterMap.add_trace(go.Scattermapbox(visible=True,                               # Make sure all points are visible
                                              name=patrolName,                            # Value of NIBRS for legend corrlation
                                              mode="markers",                             # Set mode to markers
                                              lat=patrol_District_df['Geo-Latitude'],     # Latitude Points
                                              lon=patrol_District_df['Geo-Longitude'],    # Longitude Points
                                              marker=dict(color=f'{markerColors[i]}')))   # Set the color of each point
        
        # Add to counter
        i = i+1

    # Update ScatterMap Layout
    scatterMap.update_layout(height= 750,                                                               # Height of Subplot
                             width= 1300,                                                               # Width of Subplot
                             margin={"r":0,"t":0,"l":0,"b":0},                                          # Margin values right, top, left, bottom                  
                             autosize=True,                                                             # Set Autosize of charts to True or False
                             template='plotly_dark',                                                    # Template of Subplot
                             legend=dict(title=f'{patrolColumn}'),                                      # Legend title
                             mapbox=dict(zoom=zoomLevel,                                                # Zoom of Map
                                         pitch=0,                                                       # Pitch of Map
                                         bearing=0,                                                     # Bearing of Map
                                         style=map_theme,                                               # Theme of Scatter Map
                                         center=dict(lat=np.mean(stl_patrol_df['Geo-Latitude']),        # Mean of Area Latitudes
                                                     lon=np.mean(stl_patrol_df['Geo-Longitude']))))     # Mean of Area Longitudes  
    
    # Add Geographical Data Quality Annotation
    scatterMap.add_annotation(dict(y=0.01,                                              # y position 
                                    x=0.01,                                             # X position 
                                    yref="paper",                                       # set y reference to paper
                                    xref="paper",                                       # Set X reference to paper
                                    xanchor='left',                                     # Set x anchor
                                    showarrow=False,                                    # Don't show arrow
                                    textangle=0,                                        # Angle of Title
                                    font=dict(color='grey',                             # Font color
                                                size=8),                                # Font size
                                    text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation

    # Create Path information
    coverImagePath = f'ScatterMap-2023-(Patrol-Districts).html'
    
    # Set the full path
    finalPath = os.path.join(areaPath,coverImagePath)

    # Save HTML
    scatterMap.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Scatter Maps are Created for {patrolColumn}!')

# Create list of all NeighborHoods
stl_patrol_df = stl_df.loc[(stl_df['Geo-Neighborhood'].notnull()) & (stl_df['Nei-Patrol-District'].notnull())]

# Create Scatter Maps
createPatrolDistrictScatterMap(dataFrame=stl_patrol_df,              # DataFrame                 
                               patrolColumn='Nei-Patrol-District',   # Patrol Neighborhood column 
                               zoomLevel=10.5,                       # Zoom Level value  
                               areaPath='Patrol Districts/')         # Directory location                        

[SUCCESS] Scatter Maps are Created for Nei-Patrol-District!


### Patrol Bar Charts

In [6]:
def createPatrolBarChart(dataFrame, areaColumn, nibrsColumn, areaPath):
    '''
    Title: Create Patrol Bar Chart
    Description: This function creates a barchart of all the crime for that given area
    with the following NIBRS column.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        dataFrame = dataFrame.loc[(stl_df['Geo-Neighborhood'].notnull()) & (stl_df['Nei-Patrol-District'].notnull())]
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create Bar Chart
        barChart = go.Figure(go.Bar(name=f'Total Crime by {nibrsColumn}',   # Chart Title
                             text=area_vc["count"],                         # Add Data Labels to bars
                             orientation='h',                               # Orientation of Chart
                             y=area_vc[nibrsColumn],                        # y Values
                             x=area_vc['count'],                            # x values
                             showlegend=False,                              # Do not show legend
                             marker=dict(color=area_vc['count'])))          # Continuous colors based on amount

        # Update BarChart Layout
        barChart.update_layout(height= 750,                                                 # Height of Subplot
                               width= 1300,                                                 # Width of Subplot
                               margin={"r":0,"t":70,"l":0,"b":75},                          # Margin values right, top, left, bottom    
                               title=dict(text=f'2023 {area} {nibrsColumn} Crime Totals',   # Title of BarChart
                                          font=dict(size=30)),                              # Title Font size
                               autosize=True,                                               # Set Autosize of charts to True or False
                               template='plotly_dark',                                      # Template of Subplot
                               yaxis=dict(autorange="reversed",                             # Reverse Y-axis order
                                          title=dict(text=f'{nibrsColumn}',                 # Y-axis title
                                                     font=dict(size=20))),                  # Y-axis title font size
                               xaxis=dict(title=dict(text=f'Amount',                        # X-axis title
                                                     font=dict(size=20))))                  # X-axis title font size
        
        # Add Totla Crime Stats
        barChart.add_annotation(dict(y=1.05,                                    # y position 
                                     x=0.78,                                    # X position 
                                     yref="paper",                              # set y reference to paper
                                     xref="paper",                              # Set X reference to paper
                                     xanchor='left',                            # Set x anchor
                                     showarrow=False,                           # Don't show arrow
                                     textangle=0,                               # Angle of Title
                                     font=dict(color='white',                   # Font color
                                               size=20),                        # Font size
                                     text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'BarChart-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}/'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        barChart.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn}!')

# Create & Save BarChart for each Patrol District
createPatrolBarChart(dataFrame=stl_df,                   # DataFrame                                   
                     areaColumn='Nei-Patrol-District',   # Area Column                                   
                     nibrsColumn='NIBRS-Offense',        # NIBRS Column                               
                     areaPath='Patrol Districts/')       # Directory Path           

createPatrolBarChart(dataFrame=stl_df,                       # DataFrame                                   
                     areaColumn='Nei-Patrol-District',       # Area Column                                   
                     nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                               
                     areaPath='Patrol Districts/')           # Directory Path                                         

[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District!
[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District!


In [8]:
def createPatrolDistrictsBarChart(dataFrame,nibrsColumn, areaPath):
    '''
    Title: Create patrol Districts Bar Chart
    Description: This function creates a barchart of all the crime for that given area
    with the following NIBRS column.
    dataFrame: The dataframe in which all data is found. 
    nibrsColumn: The column of NIBRS data!
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    stl_patrol_df = dataFrame

    # Value Count Lists
    patrol_vc = pd.DataFrame(stl_patrol_df['Nei-Patrol-District'].value_counts().reset_index())

    # Create Bar Chart
    barChart = go.Figure(go.Bar(name=f'Total Crime by {nibrsColumn}',   # Chart Title
                            text=patrol_vc["count"],                         # Add Data Labels to bars
                            orientation='h',                               # Orientation of Chart
                            y=patrol_vc['Nei-Patrol-District'],                        # y Values
                            x=patrol_vc['count'],                            # x values
                            showlegend=False,                              # Do not show legend
                            marker=dict(color=patrol_vc['count'])))          # Continuous colors based on amount

    # Update BarChart Layout
    barChart.update_layout(height= 750,                                                 # Height of Subplot
                            width= 1300,                                                 # Width of Subplot
                            margin={"r":0,"t":70,"l":0,"b":75},                          # Margin values right, top, left, bottom    
                            title=dict(text=f'2023 Patrol District Crime Totals',   # Title of BarChart
                                        font=dict(size=30)),                              # Title Font size
                            autosize=True,                                               # Set Autosize of charts to True or False
                            template='plotly_dark',                                      # Template of Subplot
                            yaxis=dict(autorange="reversed",                             # Reverse Y-axis order
                                        title=dict(text=f'{nibrsColumn}',                 # Y-axis title
                                                    font=dict(size=20))),                  # Y-axis title font size
                            xaxis=dict(title=dict(text=f'Amount',                        # X-axis title
                                                    font=dict(size=20))))                  # X-axis title font size
    
    # Add Totla Crime Stats
    barChart.add_annotation(dict(y=1.05,                                    # y position 
                                    x=0.78,                                    # X position 
                                    yref="paper",                              # set y reference to paper
                                    xref="paper",                              # Set X reference to paper
                                    xanchor='left',                            # Set x anchor
                                    showarrow=False,                           # Don't show arrow
                                    textangle=0,                               # Angle of Title
                                    font=dict(color='white',                   # Font color
                                            size=20),                        # Font size
                                    text=f"Total Crimes: {len(stl_patrol_df)}"))     # Title of Annotation

    # Create Path information
    coverImagePath = f'BarChart-2023-(Patrol Districts).html'

    # Set the full path
    finalPath = os.path.join(areaPath,coverImagePath)

    # Save HTML
    barChart.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] BarCharts made for Patrols!')

# Create & Save General Patrol District BarCharts
createPatrolDistrictsBarChart(dataFrame=stl_patrol_df,        # DataFrame
                              nibrsColumn='NIBRS-Offense',    # NIBRS Column
                              areaPath='Patrol Districts/')   # Directory Path                                                 

[SUCCESS] BarCharts made for Patrols!


### Patrol Density Charts

In [11]:
def createPatrolDensityMap(dataFrame, areaColumn, zoomLevel, areaPath):
    '''
    Title: Create Patrol Density Map
    Description: This function will create Area Denisty Maps to show where 
    crime is most common, but correlating with the other crime around them.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Density Plot
        densityPlot = go.Figure(go.Densitymapbox(name='Crime Density Map',   # Denisty Map Name             
                                                 lat=area_latitudes,         # Latitude values          
                                                 lon=area_longitudes,        # Longitude values       
                                                 radius=3))                  # Radius

        # Create Layout
        densityPlot.update_layout(height= 750,                                              # Height of Subplot
                                  width= 1300,                                                # Width of Subplot
                                  margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                  autosize=True,                                              # Set Autosize of charts to True or False
                                  template='plotly_dark',                                     # Template of Subplot
                                  mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                              pitch=0,                                        # Pitch of Map
                                              bearing=0,                                      # Bearing of Map
                                              style=map_theme,                                # Theme of Scatter Map
                                              center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                          lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set Path Information
        coverImagePath = f'DensityMap-2023-{area}.html'
        areaFullPath = f'{areaPath}{area}'

        # Set final path to save
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save image
        densityPlot.write_html(finalPath)
    
    # Show successful creation
    print(f'[SUCCESS] Area Density Maps are Created for {areaColumn}!')

# Create & Save Density Maps
createPatrolDensityMap(dataFrame=stl_df,                 # DataFrame
                     areaColumn='Nei-Patrol-District',   # Column of Area
                     zoomLevel=11.5,                     # Zoom Level        
                     areaPath='Patrol Districts/')       # Path to directory

[SUCCESS] Area Density Maps are Created for Nei-Patrol-District!


### Patrol Combo Charts

In [5]:
def createPatrolScatterBarCombo(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Patrol Combo Chart
    Descritption: This will create a scatter map and a barchart giving brief info about
    the area!
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: The column of NIBRS data that you want to make a chart with.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        area_vc_mappings = dict()

        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'
        
        # Create Subplot Layout
        scatterBarCombo = make_subplots(rows=1,                                                   # Number of Rows
                                        cols=2,                                                   # Number of columns
                                        horizontal_spacing=.28,                                   # Horizontal Spacing between columns
                                        column_widths=[0.5, 0.5],                                 # Column Widths max = 1.0
                                        specs=[[{"type": "scattermapbox"},{"type": "bar"}]])      # Specify Chart Types
                                                  
        # Create Geographic Map
        scatterBarCombo.add_scattermapbox(row=1,                                                    # Row in subplot
                                          col=1,                                                    # Column in subplot
                                          name=f'{areaColumn} Crime Points',                        # Chart Name
                                          lat=area_latitudes,                                       # Latitudes
                                          lon=area_longitudes,                                      # Longitudes
                                          showlegend=False,                                         # Do not show legend 
                                          marker=dict(color=area_df['NIBRS-Map']),                  # Set Markder color for each point
                                          customdata=area_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                          hovertemplate="<br>".join([area_hover, nibrs_hover]))     # Hover Information Template     

        # Create Bar Chart
        scatterBarCombo.add_bar(row=1,                                                                # Row in Subplot
                             col=2,                                                                   # Column in subplot
                             name=f'Total Crime by {nibrsColumn}',                                    # Chart Title
                             text=area_vc["count"],                                                   # Add Data Labels to bars
                             orientation='h',                                                         # Orientation of Chart
                             y=area_vc[nibrsColumn],                                                  # y Values
                             x=area_vc['count'],                                                      # x values
                             showlegend=False,                                                        # Do not show legent
                             marker=dict(color=area_vc['count']),                                     # Marker Color
                             customdata=area_vc[['count',nibrsColumn]],                               # Custom Data Columns for Hover
                             hovertemplate="<br>".join([nibrs_hover,"Amount: %{customdata[0]}"]))     # Hover Information Template

        # Reverse order of values for Y-axis
        scatterBarCombo.update_yaxes(row=1,                    # Row in Subplot
                                     col=2,                    # Column in Subplot
                                     autorange="reversed")     # Reverse order of bars

        # Create Layout for Subplot
        scatterBarCombo.update_layout(height= 750,                                                # Height of Subplot
                                      width= 1300,                                                # Width of Subplot
                                      margin={"r":20,"t":60,"l":20,"b":10},                       # Margin values right, top, left, bottom                  
                                      autosize=True,                                              # Set Autosize of charts to True or False
                                      template='plotly_dark',                                     # Template of Subplot
                                      title=dict(text=f'{area} {nibrsColumn} Overview',           # Title of Subplot
                                                 font=dict(color="White",                         # Font color
                                                           size=25)),                             # Font size
                                      font=dict(color='White',                                    # Font Color
                                                size=15),                                         # Font Size
                                      mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                                  pitch=0,                                        # Pitch of Map
                                                  bearing=0,                                      # Bearing of Map
                                                  style=map_theme,                                # Theme of Scatter Map
                                                  center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                              lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Add Geographical Data Quality Annotation
        scatterBarCombo.add_annotation(dict(y=0.01,                                             # y position 
                                            x=0.01,                                             # X position 
                                            yref="paper",                                       # set y reference to paper
                                            xref="paper",                                       # Set X reference to paper
                                            xanchor='left',                                     # Set x anchor
                                            showarrow=False,                                    # Don't show arrow
                                            textangle=0,                                        # Angle of Title
                                            font=dict(color='grey',size=8),                     # Font color, size
                                            text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
        
        # Add Totla Crime Stat
        scatterBarCombo.add_annotation(dict(y=1.05,                                    # y position 
                                            x=0.75,                                    # X position 
                                            yref="paper",                              # set y reference to paper
                                            xref="paper",                              # Set X reference to paper
                                            xanchor='left',                            # Set x anchor
                                            showarrow=False,                           # Don't show arrow
                                            textangle=0,                               # Angle of Title
                                            font=dict(color='white',                   # Font color
                                                      size=20),                        # Font size
                                            text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMapBarChartCombo-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterBarCombo.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn} & {nibrsColumn}!')
        
# Create Scatter Map & Barchart Combos    
createPatrolScatterBarCombo(dataFrame=stl_df,                   # DataFrame                   
                            areaColumn='Nei-Patrol-District',   # Area Column                              
                            nibrsColumn='NIBRS-Offense',        # NIBRS Column                          
                            zoomLevel=11.5,                     # Zoom Level value           
                            areaPath='Patrol Districts/')          # Directory location           

createPatrolScatterBarCombo(dataFrame=stl_df,                       # DataFrame                   
                            areaColumn='Nei-Patrol-District',       # Area Column                              
                            nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                          
                            zoomLevel=11.5,                         # Zoom Level value           
                            areaPath='Patrol Districts/')              # Directory location  


[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Nei-Patrol-District & NIBRS-Offense-Category!


## County & City Visualizations

### CC Pictures

In [ ]:
def createCountyCityPictures(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create County City Pictures
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: The column name of the type of nibrs information that would be mapped.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create dictionary
        area_vc_mappings = dict()

        # Reverse the order of dataframe
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{areaColumn} Crime Points',   # Chart Name
                             lat=area_latitudes,                                  # Latitudes
                             lon=area_longitudes,                                 # Longitudes
                             showlegend=False,                                    # Don't show legends
                             marker=dict(color=area_df['NIBRS-Map'])))            # Hover Information Template     

        # Create Layout
        mapPlot.update_layout(height= 750,                                                # Height of Subplot
                              width= 1300,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        
        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        coverImagePath = f'CoverImage-2023-{area}.jpeg'
        areaFullPath = f'{areaPath}'

        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save Image
        mapPlot.write_image(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Pictures Created for {areaColumn}!')

# Create Scatter Maps
createCountyCityPictures(dataFrame=stl_city_df,         # DataFrame       
                         areaColumn='Geo-County',       # Column of Area            
                         nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column                           
                         zoomLevel=10.7,                # Zoom Level           
                         areaPath='City/')              # Path to directory          

createCountyCityPictures(dataFrame=stl_county_df,       # DataFrame        
                         areaColumn='Geo-County',       # Column of Area            
                         nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column                     
                         zoomLevel=9.8,                 # Zoom Level                  
                         areaPath='County/')            # Path to directory     


### CC Density Maps

In [20]:
def createCountyCityDensityMap(dataFrame, areaColumn, zoomLevel, areaPath):
    '''
    Title: Create County & City Denisty Map
    Description: This function will create Area Denisty Maps to show where 
    crime is most common, but correlating with the other crime around them.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Density Plot
        densityPlot = go.Figure(go.Densitymapbox(name='Crime Density Map',   # Denisty Map Name             
                                                 lat=area_latitudes,         # Latitude values          
                                                 lon=area_longitudes,        # Longitude values       
                                                 radius=2))                 # Radius

        # Create Layout
        densityPlot.update_layout(height= 750,                                              # Height of Subplot
                                  width= 1300,                                                # Width of Subplot
                                  margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                  autosize=True,                                              # Set Autosize of charts to True or False
                                  template='plotly_dark',                                     # Template of Subplot
                                  mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                              pitch=0,                                        # Pitch of Map
                                              bearing=0,                                      # Bearing of Map
                                              style=map_theme,                                # Theme of Scatter Map
                                              center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                          lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set Path Information
        coverImagePath = f'DensityMap-2023-{area}.html'
        areaFullPath = f'{areaPath}'

        # Set final path to save
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save image
        densityPlot.write_html(finalPath)
    
    # Show successful creation
    print(f'[SUCCESS] Area Density Maps are Created for {area}!')

# Create & Save Density Maps
createCountyCityDensityMap(dataFrame=stl_city_df,     # DataFrame
                           areaColumn='Geo-County',   # Column of Area
                           zoomLevel=11,              # Zoom Level        
                           areaPath='City/')          # Path to directory

createCountyCityDensityMap(dataFrame=stl_county_df,   # DataFrame
                           areaColumn='Geo-County',   # Column of Area
                           zoomLevel=10,              # Zoom Level    
                           areaPath='County/')        # Path to directory

[SUCCESS] Area Density Maps are Created for Saint-Louis-City!
[SUCCESS] Area Density Maps are Created for Saint-Louis-County!


### CC Scatter Maps

In [18]:
def createCountyCityScatterMap(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create County & City Scatter Maps
    Description: This function creates a scatter map of all the latitudes and longitudes
    correlating to the color of how many crime points there are.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())
        area_vc_mappings = dict()
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)

        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'

        # Create Scatter Map Base Plot
        scatterMap = go.Figure(go.Scattermapbox())

        # Create list of Colors that correlate to continuous color values like other charts
        markerColors = px.colors.sample_colorscale('plasma',                                               # Color palette list
                                                   samplepoints=area_vc['count']/area_vc['count'].max())   # The range of point from min-max

        # Create Counter
        i=0

        # Add each unique NIBRS column value to the scatter map in correlating colors
        for nibrsName in area_vc[nibrsColumn].unique():
            
            # Create that nibrs column value as it owns dataframe
            nibs_df = area_df[area_df[nibrsColumn] == nibrsName]
            
            # Create each trace of Points
            scatterMap.add_trace(go.Scattermapbox(visible=True,                                             # Make sure all points are visible
                                                  name=nibrsName,                                           # Value of NIBRS for legend corrlation
                                                  mode="markers",                                           # Set mode to markers
                                                  lat=nibs_df['Geo-Latitude'],                              # Latitude Points
                                                  lon=nibs_df['Geo-Longitude'],                             # Longitude Points
                                                  marker=dict(color=f'{markerColors[i]}'),                  # Set the color of each point
                                                  customdata=nibs_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                                  hovertemplate="<br>".join([area_hover, nibrs_hover])))    # Hover value structure
            
            # Add to counter
            i = i+1

        # Update ScatterMap Layout
        scatterMap.update_layout(height= 750,                                                   # Height of Subplot
                                 width= 1300,                                                   # Width of Subplot
                                 margin={"r":0,"t":0,"l":0,"b":0},                              # Margin values right, top, left, bottom                  
                                 autosize=True,                                                 # Set Autosize of charts to True or False
                                 template='plotly_dark',                                        # Template of Subplot
                                 legend=dict(title=f'{nibrsColumn}'),                           # Legend title
                                 mapbox=dict(zoom=zoomLevel,                                    # Zoom of Map
                                                pitch=0,                                        # Pitch of Map
                                                bearing=0,                                      # Bearing of Map
                                                style=map_theme,                                # Theme of Scatter Map
                                                center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                            lon=np.mean(area_longitudes))))     # Mean of Area Longitudes    
        
        # Add Geographical Data Quality Annotation
        scatterMap.add_annotation(dict(y=0.01,                                             # y position 
                                       x=0.01,                                             # X position 
                                       yref="paper",                                       # set y reference to paper
                                       xref="paper",                                       # Set X reference to paper
                                       xanchor='left',                                     # Set x anchor
                                       showarrow=False,                                    # Don't show arrow
                                       textangle=0,                                        # Angle of Title
                                       font=dict(color='grey',                             # Font color
                                                 size=8),                                  # Font size
                                       text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
    

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMap-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterMap.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {area} & {nibrsColumn}!')

# Create Scatter Maps
createCountyCityScatterMap(dataFrame=stl_city_df,                  # DataFrame                 
                           areaColumn='Geo-County',                # Area Column           
                           nibrsColumn='NIBRS-Offense',            # NIBRS Column                 
                           zoomLevel=10.7,                         # Zoom Level value  
                           areaPath='City/')                       # Directory location        

createCountyCityScatterMap(dataFrame=stl_county_df,                # DataFrame                              
                           areaColumn='Geo-County',                # Area Column                          
                           nibrsColumn='NIBRS-Offense',            # NIBRS Column                                      
                           zoomLevel=9.8,                          # Zoom Level value             
                           areaPath='County/')                     # Directory location 

createCountyCityScatterMap(dataFrame=stl_city_df,                  # DataFrame                 
                           areaColumn='Geo-County',                # Area Column           
                           nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                 
                           zoomLevel=10.7,                         # Zoom Level value  
                           areaPath='City/')                       # Directory location        

createCountyCityScatterMap(dataFrame=stl_county_df,                # DataFrame                              
                           areaColumn='Geo-County',                # Area Column                          
                           nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                                      
                           zoomLevel=9.8,                          # Zoom Level value             
                           areaPath='County/')                     # Directory location                   

[SUCCESS] Area Scatter Maps are Created for Saint-Louis-City & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-County & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-City & NIBRS-Offense-Category!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-County & NIBRS-Offense-Category!


### CC Bar Charts

In [19]:
def createCountyCityBarChart(dataFrame, areaColumn, nibrsColumn, areaPath):
    '''
    Title: Create County & City Bar Charts
    Description: This function creates a barchart of all the crime for that given area
    with the following NIBRS column.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: NIBRS column to indclude.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create Bar Chart
        barChart = go.Figure(go.Bar(name=f'Total Crime by {nibrsColumn}',   # Chart Title
                             text=area_vc["count"],                         # Add Data Labels to bars
                             orientation='h',                               # Orientation of Chart
                             y=area_vc[nibrsColumn],                        # y Values
                             x=area_vc['count'],                            # x values
                             showlegend=False,                              # Do not show legend
                             marker=dict(color=area_vc['count'])))          # Continuous colors based on amount

        # Update BarChart Layout
        barChart.update_layout(height= 750,                                                 # Height of Subplot
                               width= 1300,                                                 # Width of Subplot
                               margin={"r":0,"t":70,"l":0,"b":75},                          # Margin values right, top, left, bottom    
                               title=dict(text=f'2023 {area} {nibrsColumn} Crime Totals',   # Title of BarChart
                                          font=dict(size=30)),                              # Title Font size
                               autosize=True,                                               # Set Autosize of charts to True or False
                               template='plotly_dark',                                      # Template of Subplot
                               yaxis=dict(autorange="reversed",                             # Reverse Y-axis order
                                          title=dict(text=f'{nibrsColumn}',                 # Y-axis title
                                                     font=dict(size=20))),                  # Y-axis title font size
                               xaxis=dict(title=dict(text=f'Amount',                        # X-axis title
                                                     font=dict(size=20))))                  # X-axis title font size
        
        # Add Totla Crime Stats
        barChart.add_annotation(dict(y=1.05,                                    # y position 
                                     x=0.78,                                    # X position 
                                     yref="paper",                              # set y reference to paper
                                     xref="paper",                              # Set X reference to paper
                                     xanchor='left',                            # Set x anchor
                                     showarrow=False,                           # Don't show arrow
                                     textangle=0,                               # Angle of Title
                                     font=dict(color='white',                   # Font color
                                               size=20),                        # Font size
                                     text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'BarChart-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        barChart.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {area} & {nibrsColumn}!')

# Create & Save Density Maps
createCountyCityBarChart(dataFrame=stl_city_df,           # DataFrame                                   
                           areaColumn='Geo-County',       # Area Column                                   
                           nibrsColumn='NIBRS-Offense',   # NIBRS Column                               
                           areaPath='City/')              # Directory Path                                          

createCountyCityBarChart(dataFrame=stl_county_df,         # DataFrame           
                           areaColumn='Geo-County',       # Area Column          
                           nibrsColumn='NIBRS-Offense',   # NIBRS Column                           
                           areaPath='County/')            # Directory Path

createCountyCityBarChart(dataFrame=stl_city_df,                    # DataFrame                                   
                           areaColumn='Geo-County',                # Area Column                                   
                           nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                               
                           areaPath='City/')                       # Directory Path                                          

createCountyCityBarChart(dataFrame=stl_county_df,                  # DataFrame           
                           areaColumn='Geo-County',                # Area Column          
                           nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                           
                           areaPath='County/')                     # Directory Path           

[SUCCESS] Area Scatter Maps are Created for Saint-Louis-City & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-County & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-City & NIBRS-Offense-Category!
[SUCCESS] Area Scatter Maps are Created for Saint-Louis-County & NIBRS-Offense-Category!


## Area Visualizations

### Area Pictures

In [ ]:
def createAreaPictures(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Area Map Picture
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: The column name of the type of nibrs information that would be mapped.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create dictionary
        area_vc_mappings = dict()

        # Reverse the order of dataframe
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{areaColumn} Crime Points',   # Chart Name
                             lat=area_latitudes,                                  # Latitudes
                             lon=area_longitudes,                                 # Longitudes
                             showlegend=False,                                    # Don't show legends
                             marker=dict(color=area_df['NIBRS-Map'])))            # Hover Information Template     

        # Create Layout
        mapPlot.update_layout(height= 900,                                                # Height of Subplot
                              width= 1400,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        
        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        coverImagePath = f'CoverImage-2023-{area}.jpeg'
        areaFullPath = f'{areaPath}{area}'

        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save Image
        mapPlot.write_image(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Pictures Created for {areaColumn}!')

# Create Cover Images for each Town, Village & Neighborhood
createAreaPictures(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Town',         # Column of Area
                   nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column
                   zoomLevel=12.5,                # Zoom Level        
                   areaPath='Towns/')             # Path to directory

createAreaPictures(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Village',      # Column of Area
                   nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column
                   zoomLevel=12.5,                # Zoom Level    
                   areaPath='Villages/')          # Path to directory

createAreaPictures(dataFrame=stl_df,                # DataFrame
                   areaColumn='Geo-Neighborhood',   # Column of Area
                   nibrsColumn='NIBRS-Offense',     # NIBRS Crime Column
                   zoomLevel=14,                    # Zoom Level    
                   areaPath='Neighborhoods/')       # Path to directory  

### Area Density Maps

In [17]:
def createAreaDensityMap(dataFrame, areaColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Denisty Map
    Description: This function will create Area Denisty Maps to show where 
    crime is most common, but correlating with the other crime around them.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Density Plot
        densityPlot = go.Figure(go.Densitymapbox(name='Crime Density Map',   # Denisty Map Name             
                                                 lat=area_latitudes,         # Latitude values          
                                                 lon=area_longitudes,        # Longitude values       
                                                 radius=10))                 # Radius

        # Create Layout
        densityPlot.update_layout(height= 750,                                              # Height of Subplot
                                  width= 1300,                                                # Width of Subplot
                                  margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                  autosize=True,                                              # Set Autosize of charts to True or False
                                  template='plotly_dark',                                     # Template of Subplot
                                  mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                              pitch=0,                                        # Pitch of Map
                                              bearing=0,                                      # Bearing of Map
                                              style=map_theme,                                # Theme of Scatter Map
                                              center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                          lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set Path Information
        coverImagePath = f'DensityMap-2023-{area}.html'
        areaFullPath = f'{areaPath}{area}'

        # Set final path to save
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save image
        densityPlot.write_html(finalPath)
    
    # Show successful creation
    print(f'[SUCCESS] Area Density Maps are Created for {areaColumn}!')

# Create & Save Density Maps
createAreaDensityMap(dataFrame=stl_df,        # DataFrame
                     areaColumn='Geo-Town',   # Column of Area
                     zoomLevel=12.5,          # Zoom Level        
                     areaPath='Towns/')       # Path to directory

createAreaDensityMap(dataFrame=stl_df,           # DataFrame
                     areaColumn='Geo-Village',   # Column of Area
                     zoomLevel=12.5,             # Zoom Level    
                     areaPath='Villages/')       # Path to directory

createAreaDensityMap(dataFrame=stl_df,                # DataFrame
                     areaColumn='Geo-Neighborhood',   # Column of Area
                     zoomLevel=14,                    # Zoom Level    
                     areaPath='Neighborhoods/')       # Path to directory  

[SUCCESS] Area Density Maps are Created for Geo-Town!
[SUCCESS] Area Density Maps are Created for Geo-Village!
[SUCCESS] Area Density Maps are Created for Geo-Neighborhood!


### Area Scatter Maps

In [ ]:
# Show all color swatches for continuous colors
fig = px.colors.sequential.swatches_continuous() 
fig.show() 

In [33]:
def createAreaScatterMap(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Scatter Map
    Description: This function creates a scatter map of all the latitudes and longitudes
    correlating to the color of how many crime points there are.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())
        area_vc_mappings = dict()
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)

        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'

        # Create Scatter Map Base Plot
        scatterMap = go.Figure(go.Scattermapbox())

        # Create list of Colors that correlate to continuous color values like other charts
        markerColors = px.colors.sample_colorscale('plasma',                                             # Color palette list
                                                   samplepoints=area_vc['count']/area_vc['count'].max())   # The range of point from min-max

        # Create Counter
        i=0

        # Add each unique NIBRS column value to the scatter map in correlating colors
        for nibrsName in area_vc[nibrsColumn].unique():
            
            # Create that nibrs column value as it owns dataframe
            nibs_df = area_df[area_df[nibrsColumn] == nibrsName]
            
            # Create each trace of Points
            scatterMap.add_trace(go.Scattermapbox(visible=True,                                             # Make sure all points are visible
                                                  name=nibrsName,                                           # Value of NIBRS for legend corrlation
                                                  mode="markers",                                           # Set mode to markers
                                                  lat=nibs_df['Geo-Latitude'],                              # Latitude Points
                                                  lon=nibs_df['Geo-Longitude'],                             # Longitude Points
                                                  marker=dict(color=f'{markerColors[i]}'),                  # Set the color of each point
                                                  customdata=nibs_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                                  hovertemplate="<br>".join([area_hover, nibrs_hover])))    # Hover value structure
            
            # Add to counter
            i = i+1

        # Update ScatterMap Layout
        scatterMap.update_layout(height= 750,                                                   # Height of Subplot
                                 width= 1300,                                                   # Width of Subplot
                                 margin={"r":0,"t":0,"l":0,"b":0},                              # Margin values right, top, left, bottom                  
                                 autosize=True,                                                 # Set Autosize of charts to True or False
                                 template='plotly_dark',                                        # Template of Subplot
                                 legend=dict(title=f'{nibrsColumn}'),                           # Legend title
                                 mapbox=dict(zoom=zoomLevel,                                    # Zoom of Map
                                                pitch=0,                                        # Pitch of Map
                                                bearing=0,                                      # Bearing of Map
                                                style=map_theme,                                # Theme of Scatter Map
                                                center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                            lon=np.mean(area_longitudes))))     # Mean of Area Longitudes    # Mean of Area Longitudes
        
        # Add Geographical Data Quality Annotation
        scatterMap.add_annotation(dict(y=0.01,                                             # y position 
                                       x=0.01,                                             # X position 
                                       yref="paper",                                       # set y reference to paper
                                       xref="paper",                                       # Set X reference to paper
                                       xanchor='left',                                     # Set x anchor
                                       showarrow=False,                                    # Don't show arrow
                                       textangle=0,                                        # Angle of Title
                                       font=dict(color='grey',                             # Font color
                                                 size=8),                                  # Font size
                                       text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
    

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMap-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterMap.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn} & {nibrsColumn}!')

# Create Scatter Maps
createAreaScatterMap(dataFrame=stl_df,              # DataFrame
                     areaColumn='Geo-Town',         # Area Column
                     nibrsColumn='NIBRS-Offense',   # NIBRS Column
                     zoomLevel=12.5,                # Zoom Level value
                     areaPath='Towns/')             # Directory location

createAreaScatterMap(dataFrame=stl_df,              # DataFrame        
                     areaColumn='Geo-Village',      # Area Column        
                     nibrsColumn='NIBRS-Offense',   # NIBRS Column             
                     zoomLevel=12.5,                # Zoom Level value               
                     areaPath='Villages/')          # Directory location           

createAreaScatterMap(dataFrame=stl_df,                # DataFrame          
                     areaColumn='Geo-Neighborhood',   # Area Column                  
                     nibrsColumn='NIBRS-Offense',     # NIBRS Column        
                     zoomLevel=14,                    # Zoom Level value      
                     areaPath='Neighborhoods/')       # Directory location    

createAreaScatterMap(dataFrame=stl_df,                       # DataFrame
                     areaColumn='Geo-Town',                  # Area Column
                     nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column
                     zoomLevel=12.5,                         # Zoom Level value
                     areaPath='Towns/')                      # Directory location

createAreaScatterMap(dataFrame=stl_df,                       # DataFrame        
                     areaColumn='Geo-Village',               # Area Column        
                     nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column             
                     zoomLevel=12.5,                         # Zoom Level value               
                     areaPath='Villages/')                   # Directory location           

createAreaScatterMap(dataFrame=stl_df,                       # DataFrame          
                     areaColumn='Geo-Neighborhood',          # Area Column                  
                     nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column        
                     zoomLevel=14,                           # Zoom Level value      
                     areaPath='Neighborhoods/')              # Directory location                   

[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!
[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!


### Area BarCharts

In [32]:
def createAreaBarChart(dataFrame, areaColumn, nibrsColumn, areaPath):
    '''
    Title: Create Bar Chart
    Description: This function creates a barchart of all the crime for that given area
    with the following NIBRS column.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create Bar Chart
        barChart = go.Figure(go.Bar(name=f'Total Crime by {nibrsColumn}',   # Chart Title
                             text=area_vc["count"],                         # Add Data Labels to bars
                             orientation='h',                               # Orientation of Chart
                             y=area_vc[nibrsColumn],                        # y Values
                             x=area_vc['count'],                            # x values
                             showlegend=False,                              # Do not show legend
                             marker=dict(color=area_vc['count'])))          # Continuous colors based on amount

        # Update BarChart Layout
        barChart.update_layout(height= 750,                                                 # Height of Subplot
                               width= 1300,                                                 # Width of Subplot
                               margin={"r":0,"t":70,"l":0,"b":75},                          # Margin values right, top, left, bottom    
                               title=dict(text=f'2023 {area} {nibrsColumn} Crime Totals',   # Title of BarChart
                                          font=dict(size=30)),                              # Title Font size
                               autosize=True,                                               # Set Autosize of charts to True or False
                               template='plotly_dark',                                      # Template of Subplot
                               yaxis=dict(autorange="reversed",                             # Reverse Y-axis order
                                          title=dict(text=f'{nibrsColumn}',                 # Y-axis title
                                                     font=dict(size=20))),                  # Y-axis title font size
                               xaxis=dict(title=dict(text=f'Amount',                        # X-axis title
                                                     font=dict(size=20))))                  # X-axis title font size
        
        # Add Totla Crime Stats
        barChart.add_annotation(dict(y=1.05,                                    # y position 
                                     x=0.78,                                    # X position 
                                     yref="paper",                              # set y reference to paper
                                     xref="paper",                              # Set X reference to paper
                                     xanchor='left',                            # Set x anchor
                                     showarrow=False,                           # Don't show arrow
                                     textangle=0,                               # Angle of Title
                                     font=dict(color='white',                   # Font color
                                               size=20),                        # Font size
                                     text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'BarChart-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        barChart.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn} & {nibrsColumn}!')

# Create BarCharts
createAreaBarChart(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Town',         # Area Column           
                   nibrsColumn='NIBRS-Offense',   # NIBRS Column                  
                   areaPath='Towns/')             # Directory Path

createAreaBarChart(dataFrame=stl_df,              # DataFrame               
                   areaColumn='Geo-Village',      # Area Column                  
                   nibrsColumn='NIBRS-Offense',   # NIBRS Column                       
                   areaPath='Villages/')          # Directory Path            

createAreaBarChart(dataFrame=stl_df,                # DataFrame         
                   areaColumn='Geo-Neighborhood',   # Area Column                           
                   nibrsColumn='NIBRS-Offense',     # NIBRS Column                     
                   areaPath='Neighborhoods/')       # Directory Path     

createAreaBarChart(dataFrame=stl_df,                       # DataFrame
                   areaColumn='Geo-Town',                  # Area Column           
                   nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                  
                   areaPath='Towns/')                      # Directory Path

createAreaBarChart(dataFrame=stl_df,                       # DataFrame               
                   areaColumn='Geo-Village',               # Area Column                  
                   nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                       
                   areaPath='Villages/')                   # Directory Path            

createAreaBarChart(dataFrame=stl_df,                       # DataFrame         
                   areaColumn='Geo-Neighborhood',          # Area Column                           
                   nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                     
                   areaPath='Neighborhoods/')              # Directory Path              

[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!
[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!


### Scatter Maps & BarChart Combo

In [35]:
def createAreaScatterBarCombo(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Scatter Combo Chart
    Descritption: This will create a scatter map and a barchart giving brief info about
    the area!
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    nibrsColumn: The column of NIBRS data that you want to make a chart with.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''
    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        area_vc_mappings = dict()

        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'
        
        # Create Subplot Layout
        scatterBarCombo = make_subplots(rows=1,                                                   # Number of Rows
                                        cols=2,                                                   # Number of columns
                                        horizontal_spacing=.28,                                   # Horizontal Spacing between columns
                                        column_widths=[0.5, 0.5],                                 # Column Widths max = 1.0
                                        specs=[[{"type": "scattermapbox"},{"type": "bar"}]])      # Specify Chart Types
                                                  
        # Create Geographic Map
        scatterBarCombo.add_scattermapbox(row=1,                                                    # Row in subplot
                                          col=1,                                                    # Column in subplot
                                          name=f'{areaColumn} Crime Points',                        # Chart Name
                                          lat=area_latitudes,                                       # Latitudes
                                          lon=area_longitudes,                                      # Longitudes
                                          showlegend=False,                                         # Do not show legend 
                                          marker=dict(color=area_df['NIBRS-Map']),                  # Set Markder color for each point
                                          customdata=area_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                          hovertemplate="<br>".join([area_hover, nibrs_hover]))     # Hover Information Template     

        # Create Bar Chart
        scatterBarCombo.add_bar(row=1,                                                                   # Row in Subplot
                                col=2,                                                                   # Column in subplot
                                name=f'Total Crime by {nibrsColumn}',                                    # Chart Title
                                text=area_vc["count"],                                                   # Add Data Labels to bars
                                orientation='h',                                                         # Orientation of Chart
                                y=area_vc[nibrsColumn],                                                  # y Values
                                x=area_vc['count'],                                                      # x values
                                showlegend=False,                                                        # Do not show legent
                                marker=dict(color=area_vc['count']),                                     # Marker Color
                                customdata=area_vc[['count',nibrsColumn]],                               # Custom Data Columns for Hover
                                hovertemplate="<br>".join([nibrs_hover,"Amount: %{customdata[0]}"]))     # Hover Information Template

        # Reverse order of values for Y-axis
        scatterBarCombo.update_yaxes(row=1,                    # Row in Subplot
                                     col=2,                    # Column in Subplot
                                     autorange="reversed")     # Reverse order of bars

        # Create Layout for Subplot
        scatterBarCombo.update_layout(height= 750,                                                # Height of Subplot
                                      width= 1300,                                                # Width of Subplot
                                      margin={"r":20,"t":60,"l":20,"b":10},                       # Margin values right, top, left, bottom                  
                                      autosize=True,                                              # Set Autosize of charts to True or False
                                      template='plotly_dark',                                     # Template of Subplot
                                      title=dict(text=f'{area} {nibrsColumn} Overview',           # Title of Subplot
                                                 font=dict(color="White",                         # Font color
                                                           size=25)),                             # Font size
                                      font=dict(color='White',                                    # Font Color
                                                size=15),                                         # Font Size
                                      mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                                  pitch=0,                                        # Pitch of Map
                                                  bearing=0,                                      # Bearing of Map
                                                  style=map_theme,                                # Theme of Scatter Map
                                                  center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                              lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Add Geographical Data Quality Annotation
        scatterBarCombo.add_annotation(dict(y=0.01,                                             # y position 
                                            x=0.01,                                             # X position 
                                            yref="paper",                                       # set y reference to paper
                                            xref="paper",                                       # Set X reference to paper
                                            xanchor='left',                                     # Set x anchor
                                            showarrow=False,                                    # Don't show arrow
                                            textangle=0,                                        # Angle of Title
                                            font=dict(color='grey',size=8),                     # Font color, size
                                            text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
        
        # Add Totla Crime Stat
        scatterBarCombo.add_annotation(dict(y=1.05,                                    # y position 
                                            x=0.75,                                    # X position 
                                            yref="paper",                              # set y reference to paper
                                            xref="paper",                              # Set X reference to paper
                                            xanchor='left',                            # Set x anchor
                                            showarrow=False,                           # Don't show arrow
                                            textangle=0,                               # Angle of Title
                                            font=dict(color='white',                   # Font color
                                                      size=20),                        # Font size
                                            text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMapBarChartCombo-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterBarCombo.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn} & {nibrsColumn}!')
        
# Create Scatter Map & Barchart Combos
createAreaScatterBarCombo(dataFrame=stl_df,              # DataFrame                
                          areaColumn='Geo-Town',         # Area Column                  
                          nibrsColumn='NIBRS-Offense',   # NIBRS Column                       
                          zoomLevel=12,                  # Zoom Level value        
                          areaPath='Towns/')             # Directory location         

createAreaScatterBarCombo(dataFrame=stl_df,              # DataFrame                   
                          areaColumn='Geo-Village',      # Area Column                     
                          nibrsColumn='NIBRS-Offense',   # NIBRS Column                          
                          zoomLevel=12,                  # Zoom Level value             
                          areaPath='Villages/')          # Directory location              

createAreaScatterBarCombo(dataFrame=stl_df,                # DataFrame                   
                          areaColumn='Geo-Neighborhood',   # Area Column                              
                          nibrsColumn='NIBRS-Offense',     # NIBRS Column                          
                          zoomLevel=13.5,                  # Zoom Level value           
                          areaPath='Neighborhoods/')       # Directory location  

createAreaScatterBarCombo(dataFrame=stl_df,                       # DataFrame                
                          areaColumn='Geo-Town',                  # Area Column                  
                          nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                       
                          zoomLevel=12,                           # Zoom Level value        
                          areaPath='Towns/')                      # Directory location         

createAreaScatterBarCombo(dataFrame=stl_df,                       # DataFrame                   
                          areaColumn='Geo-Village',               # Area Column                     
                          nibrsColumn='NIBRS-Offense-Category',   # NIBRS Column                          
                          zoomLevel=12,                           # Zoom Level value             
                          areaPath='Villages/')                   # Directory location              

createAreaScatterBarCombo(dataFrame=stl_df,                         # DataFrame                   
                          areaColumn='Geo-Neighborhood',            # Area Column                              
                          nibrsColumn='NIBRS-Offense-Category',     # NIBRS Column                          
                          zoomLevel=13.5,                           # Zoom Level value           
                          areaPath='Neighborhoods/')                # Directory location  


[SUCCESS] Area Scatter Maps are Created for Geo-Town & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Geo-Village & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood & NIBRS-Offense!
[SUCCESS] Area Scatter Maps are Created for Geo-Town & NIBRS-Offense-Category!
[SUCCESS] Area Scatter Maps are Created for Geo-Village & NIBRS-Offense-Category!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood & NIBRS-Offense-Category!


## Ranking Charts

In [14]:
def createRankingBarChart(dataFrame, area_list, areaPath):
    '''
    Title: Create Ranking Chart
    Description: This function creates a barchart of all the crime for each major Saint 
    Louis area seperated by type of place. Either Town, Village, or Neighborhood!
    dataFrame: The dataframe in which all data is found. 
    area_list: The list of areas to include.
    areaPath: Path to the folder to store the results.
    '''

    # For each Neighborhood in the list
    for area in area_list:

        # Make into more readable format
        areaName = area[4:] +'s'

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[area].notnull()]
        # Value Count Lists
        
        area_vc = pd.DataFrame(area_df[area].value_counts().reset_index())

        # Create Bar Chart
        barChart = go.Figure(go.Bar(name=f'Rank ',                          # Chart Name
                             text=area_vc["count"],                         # Add Data Labels to bars
                             orientation='h',                               # Orientation of Chart
                             y=area_vc[area],                               # y Values
                             x=area_vc['count'],                            # x values
                             showlegend=False,                              # Do not show legend
                             marker=dict(color=area_vc['count'])))          # Continuous colors based on amount

        # Update BarChart Layout
        barChart.update_layout(height=750,                                                           # Height of Subplot
                               width=1300,                                                           # Width of Subplot
                               margin={"r":0,"t":70,"l":0,"b":75},                                   # Margin values right, top, left, bottom    
                               title=dict(text=f'Rankings of Total Crime in St. Louis {areaName}',   # Title of BarChart
                                          font=dict(size=30)),                                       # Title Font size
                               autosize=True,                                                        # Set Autosize of charts to True or False
                               template='plotly_dark',                                               # Template of Subplot
                               yaxis=dict(autorange="reversed",                                      # Reserve Y-axis direction
                                          title=dict(text=f'{areaName}',                             # X-axis title
                                                     font=dict(size=20))),                           # Reverse Y-axis order
                               xaxis=dict(title=dict(text=f'Amount of Crime',                        # X-axis title
                                                     font=dict(size=20))))                           # X-axis title font size
        
        # Add Totla Crime Stats
        barChart.add_annotation(dict(y=1.05,                                    # y position 
                                     x=0.78,                                    # X position 
                                     yref="paper",                              # set y reference to paper
                                     xref="paper",                              # Set X reference to paper
                                     xanchor='left',                            # Set x anchor
                                     showarrow=False,                           # Don't show arrow
                                     textangle=0,                               # Angle of Title
                                     font=dict(color='white',                   # Font color
                                               size=20),                        # Font size
                                     text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'RankingChart-2023-{areaName}.html'
        areaFullPath = f'{areaPath}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        barChart.write_html(finalPath)

    print(f'[SUCCESS] Ranking BarCharts are created! {areaPath}')

# Create Ranking BarCharts
createRankingBarChart(dataFrame=stl_df,area_list=['Geo-Neighborhood'], areaPath='City')   # The DataFrame            
createRankingBarChart(dataFrame=stl_df,area_list=['Geo-Town','Geo-Village'], areaPath='County')   # The DataFrame            



[SUCCESS] Ranking BarCharts are created! City
[SUCCESS] Ranking BarCharts are created! County


## Create HTML Source code

### Create St.Louis HTML File

In [15]:
def stlWebLinks(dataFrame):
    areaList = ['Geo-Town','Geo-Village','Geo-Neighborhood']

    if os.path.exists("St-Louis-Crime.html"):
        os.remove("St-Louis-Crime.html")
    else:
        pass
        
    html_base = open("St-Louis-Crime.html", "w")
    html_base.close()
    
    html_base = open("St-Louis-Crime.html", "a")

    # Write header to file
    html_base.write(f'''<!DOCTYPE html>
<html>
<!-- Adding Poppins Fonts -->
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Poppins&display=swap" rel="stylesheet">

<!--Import Custom Stylesheet-->
<link rel="stylesheet" type="text/css" href="../../style.css" />

<!-- Import STL StyleSheet -->
<link rel="stylesheet" type="text/css" href="stl-style.css" />

<!-- Boxicons CSS -->
<link href='https://unpkg.com/boxicons@2.1.4/css/boxicons.min.css' rel='stylesheet'>

<!-- Add Scroll Reveal -->
<script src="https://unpkg.com/scrollreveal"></script>

<!-- Typing Animation -->
<script src="https://unpkg.com/typed.js@2.1.0/dist/typed.umd.js"></script>

<!-- Title -->
<title>Kyle's Portfolio Website</title>

<body>
    <!-- Header -->
    <header class="header">

        <!-- Logo -->
        <a href="/KyleHostetter.github.io-main/index.html" class="logo">Kyle's <span>Portfolio</span></a>

        <!-- NavBar -->
        <nav class="navbar">
            <a href="#Home" class="active">Home</a>
            <a href="#Overviews">Overview</a>
            <a href="#Towns">Towns</a>
            <a href="#Villages">Villages</a>
            <a href="#Neighborhoods">Neighborhoods</a>
        </nav>
    </header>
                    
    <!-- Home Section -->
    <section class="home" id="Home">
        <div class="home-content">
            <h1><span>St. Louis </span>Crime <span>Analysis</span></h1>
            <p>Here I take a deep dive into <span>Crime</span> within the <span>St. Louis</span> area.
                Using <span>machine learning</span>, <span>Geographical data</span>, and <span>NIBRS</span> crime
                standard I show what the real crime stats are for the <span>St. Louis</span> Area.
            </p>
            <!-- Social Media Icons -->
            <div class="social-media">
                <a href="https://www.facebook.com/kyle.hostetter.775">
                    <i class='bx bxl-meta'></i>
                </a>
                <a href="https://www.linkedin.com/in/kyle-hostetter-906182157/6">
                    <i class='bx bxl-linkedin'></i>
                </a>
                <a href="https://github.com/KyleHoss">
                    <i class='bx bxl-github'></i>
                </a>
            </div>
            <a href="StlCrimeAnalysis.ipynb" class="btn">Download Juypter Notebook</a>
        </div>
        <!-- Profile Picture -->
        <img class="stl-pic" src="St-Louis-Cover.jpeg" alt="St. Louis Area Crime Map">
    </section>             

    <section class="stl-port" id="Overviews">
        <h2 class="heading">Crime <span>Overviews</span></h2>
        <div class="stl-overview-port-container">
            <a href="City/City.html">
                <div class="stl-overview-port-box">
                    <img src="./City/CoverImage-2023-Saint-Louis-City.jpeg" alt="">
                    <div class="stl-port-layer">
                        <h4>St. Louis City</h4>
                    </div>
                    
                </div>
            </a>
            <a href="County/County.html">
                <div class="stl-overview-port-box">
                    <img src="./County/CoverImage-2023-Saint-Louis-County.jpeg" alt="">
                    <div class="stl-port-layer">
                        <h4>St. Louis County</h4>
                    </div>
                </div>
            </a>
            <a href="Patrol Districts/North-Patrol/North-Patrol.html">
                <div class="stl-overview-port-box">
                    <img src="Patrol Districts/North-Patrol/CoverImage-2023-North-Patrol.jpeg" alt="North Patrol Cover Image">
                    <div class="stl-port-layer">
                        <h4>North Patrol</h4>
                    </div>
                </div>
            </a>
            <a href="Patrol Districts/Central-Patrol/Central-Patrol.html">
                <div class="stl-overview-port-box">
                    <img src="Patrol Districts/Central-Patrol/CoverImage-2023-Central-Patrol.jpeg" alt="Central Patrol Cover Image">
                    <div class="stl-port-layer">
                        <h4>Central Patrol</h4>
                    </div>
                </div>
            </a>
            <a href="Patrol Districts/South-Patrol/South-Patrol.html">
                <div class="stl-overview-port-box">
                    <img src="Patrol Districts/South-Patrol/CoverImage-2023-South-Patrol.jpeg" alt="South Patrol Cover Image">
                    <div class="stl-port-layer">
                        <h4>South Patrol</h4>
                    </div>
                </div>
            </a>
        </div>
    </section>                    
''')
    
    # Close File
    html_base.close()
    
    for areaColumn in areaList:

        areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()).sort_values('Name').reset_index(drop=True)
        areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
        areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")

        areaFolder = areaColumn[4:] + 's'

        htmlSection = open("St-Louis-Crime.html", "a")

        htmlSection.write(f'''
    <!-- {areaFolder} -->
    <section class="stl-port" id="{areaFolder}">
        <h2 class="heading">St. Louis <span>{areaFolder}</span></h2>
        <div class="stl-port-container">
        ''')

        htmlSection.close()

        # Loop through Areas in Area Section
        for i in range(0,len(areaNames)):

            # Open file and set mode to append
            file1 = open("St-Louis-Crime.html", "a")
            
            areaSourceFolder = f'{areaFolder}/{areaNames["TotalFixed"][i]}'
            areaImageFile = f'{areaSourceFolder}/CoverImage-2023-{areaNames["TotalFixed"][i]}.jpeg'
            areaHtmlLocation = f'{areaSourceFolder}/{areaNames["TotalFixed"][i]}.html'

            # writing newline character
            file1.write(f'''\n            <!-- {areaNames['Name'][i]} -->
            <a href="{areaHtmlLocation}">
                <div class="stl-port-box">
                    <img src="{areaImageFile}" alt="{areaNames["Name"][i]} Cover Image">
                    <div class="stl-port-layer">
                        <h4>{areaNames['Name'][i]}</h4>
                    </div>
                </div>
            </a>''')
        # Close and save file
        file1.close()

        htmlSection = open("St-Louis-Crime.html", "a")
        htmlSection.write(f'''
        </div>
    </section>
''')
        htmlSection.close()

    # Add Footer
    html_base = open("St-Louis-Crime.html", "a")

    # Write header to file
    html_base.write(f'''
    <!-- Footer -->
    <footer class="footer">
        <div class="footer-text">
            <p>Built by Kyle J. Hostetter | Youtube Tutorials Helped!</p>
        </div>
        <div class="footer-iconTop">
            <a href="#Overview"><i class='bx bx-up-arrow-alt'></i></a>
        </div>
    </footer>
</body>

</html>
''')

    # Close File
    html_base.close()

stlWebLinks(stl_df)

### Area Web Pages

In [16]:
def AreaWebPages(dataFrame):
    areaList = ['Geo-Town','Geo-Village','Geo-Neighborhood']

    for areaColumn in areaList:

        areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()).sort_values('Name').reset_index(drop=True)
        areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
        areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")

        areaFolder = areaColumn[4:] + 's'

        # Loop through Areas in Area Section
        for i in range(0,len(areaNames)):

            areaSourceFolder = f'{areaFolder}/{areaNames["TotalFixed"][i]}'
            areaHTMLFile = f'{areaSourceFolder}/{areaNames["TotalFixed"][i]}.html'

            # Check if the area html already exists
            if os.path.exists(areaHTMLFile):
                os.remove(areaHTMLFile)
            else:
                areaHTML = open(areaHTMLFile, "w")
                areaHTML.close()
    
            areaHTML = open(areaHTMLFile, "a")
            areaHTML.write(f'''
<!DOCTYPE html>
<html>
<!-- Adding Poppins Fonts -->
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Poppins&display=swap" rel="stylesheet">

<!--Import Custom Stylesheet-->
<link rel="stylesheet" type="text/css" href="../../../../style.css" />

<!-- Import STL StyleSheet -->
<link rel="stylesheet" type="text/css" href="../../stl-style.css" />

<!-- Boxicons CSS -->
<link href='https://unpkg.com/boxicons@2.1.4/css/boxicons.min.css' rel='stylesheet'>

<!-- Add Scroll Reveal -->
<script src="https://unpkg.com/scrollreveal"></script>

<!-- Typing Animation -->
<script src="https://unpkg.com/typed.js@2.1.0/dist/typed.umd.js"></script>

<!-- Title -->
<title>Kyle's Portfolio Website</title>

<body>
    <!-- Header -->
    <header class="header">

        <!-- Logo -->
        <a href="../../../../index.html" class="logo">Kyle's <span>Portfolio</span></a>

        <!-- NavBar -->
        <nav class="navbar">
            <a href="../../St-Louis-Crime.html" class="active">Home</a>
            <a href="#Overview">Overview</a>
            <a href="#ComboChart">Combo Chart</a>
            <a href="#CrimeStat">Crime Chart</a>
            <a href="#ScatterMap">Geographical Chart</a>
            <a href="#DensityMap">Density Map</a>
        </nav>
    </header>

    <!-- Home Section -->
    <section class="stl-home" id="Overview">
        <div class="stl-home-content">
            <h1>{areaNames["TotalFixed"][i]}</h1>
            <p>
                Below are charts that give a representation to all the crime that was committed in {areaNames["Name"][i]} within 2023.
                All the charts are interactive! So look around and have fun!
            </p>
            <!-- Social Media Icons -->
            <div class="social-media">
                <a href="https://www.facebook.com/kyle.hostetter.775">
                    <i class='bx bxl-meta'></i>
                </a>
                <a href="https://www.linkedin.com/in/kyle-hostetter-906182157/6">
                    <i class='bx bxl-linkedin'></i>
                </a>
                <a href="https://github.com/KyleHoss">
                    <i class='bx bxl-github'></i>
                </a>
            </div>
            <a href="https://github.com/KyleHoss/KyleHostetter.github.io/tree/main/Projects/St-Louis-Crime/{areaFolder}/{areaNames["TotalFixed"][i]}" class="btn">Download {areaNames["Name"][i]} Visualizations</a>
        </div>
        <!-- Profile Picture -->
        <img class="stl-pic" src="CoverImage-2023-{areaNames["TotalFixed"][i]}.jpeg" alt="{areaNames["Name"][i]} Cover Image">
    </section>

    <!-- Overview -->
    <section class="stl-port" id="ComboChart">
        <h2 class="heading">Combo <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="ScatterMapBarChartCombo-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Towns -->
    <section class="stl-port" id="CrimeStat">
        <h2 class="heading">Crime <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="BarChart-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Villages -->
    <section class="stl-port" id="ScatterMap">
        <h2 class="heading">Geographical <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="ScatterMap-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Neighborhoods -->
    <section class="stl-port" id="DensityMap">
        <h2 class="heading"><span>Density</span> Map</h2>
        <div class="stl-charts">
            <iframe src="DensityMap-2023-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Footer -->
    <footer class="footer">
        <div class="footer-text">
            <p>Built by Kyle J. Hostetter | Youtube Tutorials Helped!</p>
        </div>
        <div class="footer-iconTop">
            <a href="#Overview"><i class='bx bx-up-arrow-alt'></i></a>
        </div>
    </footer>
</body>

</html>                    
''')

            # Open file and set mode to append
            areaHTML.close()

AreaWebPages(dataFrame=stl_df)

### Patrol District Web Pages

In [17]:
def PatrolWebPages(dataFrame, areaFolder):
    areaList = ['Nei-Patrol-District']

    for areaColumn in areaList:

        areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()).sort_values('Name').reset_index(drop=True)
        areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
        areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")

        # Loop through Areas in Area Section
        for i in range(0,len(areaNames)):

            areaSourceFolder = f'{areaFolder}{areaNames["TotalFixed"][i]}'
            areaHTMLFile = f'{areaSourceFolder}/{areaNames["TotalFixed"][i]}.html'

            # Check if the area html already exists
            if os.path.exists(areaHTMLFile):
                os.remove(areaHTMLFile)
            else:
                areaHTML = open(areaHTMLFile, "w")
                areaHTML.close()
    
            areaHTML = open(areaHTMLFile, "a")
            areaHTML.write(f'''
<!DOCTYPE html>
<html>
<!-- Adding Poppins Fonts -->
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Poppins&display=swap" rel="stylesheet">

<!--Import Custom Stylesheet-->
<link rel="stylesheet" type="text/css" href="../../../../style.css" />

<!-- Import STL StyleSheet -->
<link rel="stylesheet" type="text/css" href="../../stl-style.css" />

<!-- Boxicons CSS -->
<link href='https://unpkg.com/boxicons@2.1.4/css/boxicons.min.css' rel='stylesheet'>

<!-- Add Scroll Reveal -->
<script src="https://unpkg.com/scrollreveal"></script>

<!-- Typing Animation -->
<script src="https://unpkg.com/typed.js@2.1.0/dist/typed.umd.js"></script>

<!-- Title -->
<title>Kyle's Portfolio Website</title>

<body>
    <!-- Header -->
    <header class="header">

        <!-- Logo -->
        <a href="../../../../index.html" class="logo">Kyle's <span>Portfolio</span></a>

        <!-- NavBar -->
        <nav class="navbar">
            <a href="../../St-Louis-Crime.html" class="active">Home</a>
            <a href="#Overview">Overview</a>
            <a href="#ComboChart">Combo Chart</a>
            <a href="#CrimeStat">Crime Chart</a>
            <a href="#ScatterMap">Geographical Chart</a>
            <a href="#DensityMap">Density Map</a>
        </nav>
    </header>

    <!-- Home Section -->
    <section class="stl-home" id="Overview">
        <div class="stl-home-content">
            <h1>{areaNames["TotalFixed"][i]}</h1>
            <p>
                Below are charts that give a representation to all the crime that was committed in {areaNames["Name"][i]} within 2023.
                All the charts are interactive! So look around and have fun!
            </p>
            <!-- Social Media Icons -->
            <div class="social-media">
                <a href="https://www.facebook.com/kyle.hostetter.775">
                    <i class='bx bxl-meta'></i>
                </a>
                <a href="https://www.linkedin.com/in/kyle-hostetter-906182157/6">
                    <i class='bx bxl-linkedin'></i>
                </a>
                <a href="https://github.com/KyleHoss">
                    <i class='bx bxl-github'></i>
                </a>
            </div>
            <a href="https://github.com/KyleHoss/KyleHostetter.github.io/tree/main/Projects/St-Louis-Crime/Patrol%20Disticts/{areaNames["TotalFixed"][i]}" class="btn">Download {areaNames["Name"][i]} Visualizations</a>
        </div>
        <!-- Profile Picture -->
        <img class="stl-pic" src="CoverImage-2023-{areaNames["TotalFixed"][i]}.jpeg" alt="{areaNames["Name"][i]} Cover Image">
    </section>

    <!-- Overview -->
    <section class="stl-port" id="ComboChart">
        <h2 class="heading">Combo <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="ScatterMapBarChartCombo-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Towns -->
    <section class="stl-port" id="CrimeStat">
        <h2 class="heading">Crime <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="BarChart-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Villages -->
    <section class="stl-port" id="ScatterMap">
        <h2 class="heading">Geographical <span>Chart</span></h2>
        <div class="stl-charts">
            <iframe src="ScatterMap-2023-(NIBRS-Offense)-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Neighborhoods -->
    <section class="stl-port" id="DensityMap">
        <h2 class="heading"><span>Density</span> Map</h2>
        <div class="stl-charts">
            <iframe src="DensityMap-2023-{areaNames["TotalFixed"][i]}.html" frameborder="0"></iframe>
        </div>
    </section>

    <!-- Footer -->
    <footer class="footer">
        <div class="footer-text">
            <p>Built by Kyle J. Hostetter | Youtube Tutorials Helped!</p>
        </div>
        <div class="footer-iconTop">
            <a href="#Overview"><i class='bx bx-up-arrow-alt'></i></a>
        </div>
    </footer>
</body>

</html>                    
''')

            # Open file and set mode to append
            areaHTML.close()

PatrolWebPages(dataFrame=stl_df,                 # DataFrame
               areaFolder='Patrol Districts/')   # Area Folder to save HTML files